In [1]:
import pandas as pd
training_data_df = pd.read_csv('keras_train.csv')
testing_data_df = pd.read_csv('keras_test.csv')

In [2]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(testing_data_df)

print("mutiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[6],scaler.min_[6]))


mutiplying by 0.0214224507 and adding 0.000000


In [3]:
scaled_training_df = pd.DataFrame(scaled_training,columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing,columns=testing_data_df.columns.values)

scaled_training_df.to_csv("Options_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("Options_data_testing_scaled.csv", index=False)

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("Options_data_training_scaled.csv")
X = training_data_df.drop('premium',axis=1).values
Y = training_data_df[['premium']].values

Using TensorFlow backend.


In [5]:
model = Sequential()
model.add(Dense(50,input_dim=6,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer='adam')

In [6]:
# Train the model
model.fit(
    X,
    Y,
    epochs=100,
    shuffle=True,
    verbose=2
)

#Load the separate test dataset
test_data_df = pd.read_csv("Options_data_testing_scaled.csv")
X_test = test_data_df.drop('premium',axis=1).values
Y_test = test_data_df[['premium']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/100
8s - loss: 9.0673e-05
Epoch 2/100
7s - loss: 1.8535e-05
Epoch 3/100
7s - loss: 1.3472e-05
Epoch 4/100
7s - loss: 9.0348e-06
Epoch 5/100
7s - loss: 5.8720e-06
Epoch 6/100
7s - loss: 5.5534e-06
Epoch 7/100
7s - loss: 3.8747e-06
Epoch 8/100
7s - loss: 3.6130e-06
Epoch 9/100
7s - loss: 3.9448e-06
Epoch 10/100
8s - loss: 4.7889e-06
Epoch 11/100
7s - loss: 2.4346e-06
Epoch 12/100
7s - loss: 2.8602e-06
Epoch 13/100
7s - loss: 3.1716e-06
Epoch 14/100
7s - loss: 2.6709e-06
Epoch 15/100
7s - loss: 2.1492e-06
Epoch 16/100
7s - loss: 2.3516e-06
Epoch 17/100
7s - loss: 2.0371e-06
Epoch 18/100
7s - loss: 1.6509e-06
Epoch 19/100
7s - loss: 2.2029e-06
Epoch 20/100
7s - loss: 2.0024e-06
Epoch 21/100
8s - loss: 1.7188e-06
Epoch 22/100
7s - loss: 1.6902e-06
Epoch 23/100
7s - loss: 1.3861e-06
Epoch 24/100
8s - loss: 1.5721e-06
Epoch 25/100
8s - loss: 1.7220e-06
Epoch 26/100
8s - loss: 1.4113e-06
Epoch 27/100
8s - loss: 1.5883e-06
Epoch 28/100
7s - loss: 2.4205e-06
Epoch 29/100
8s - loss: 1.847

In [8]:
# Load the data we make to use to make a prediction
new_option_df = pd.read_csv("keras_evaluation.csv")
new_option_scaled = scaler.transform(new_option_df)
scaled_evaluation_df = pd.DataFrame(new_option_scaled,columns=new_option_df.columns.values)

X = scaled_evaluation_df.drop('premium',axis=1).values

prediction = model.predict(X)

#prediction = prediction[0][0]
#prediction = scaler.inverse_transform(prediction)
prediction = prediction - scaler.min_[6]
prediction = prediction / scaler.scale_[6] 

print("Predicted premium is  ${}".format(prediction))

Predicted premium is  $[[ 0.01218562]
 [ 0.01218562]
 [ 0.01218562]
 ..., 
 [ 0.51308346]
 [ 0.51308346]
 [ 0.51308346]]


In [9]:
model.save('trained_model.h5')